In [ ]:
%matplotlib notebook
import os; os.environ['INTERACTIVE']='y'
import pandas as pd
from simudo.trash.dirty_bandplot import do_band_plot
from simudo.trash.dirty_poster_plot import do_n_plot, do_j_plot
from simudo.util.plot import plt, subplots
import numpy as np

def add_jtot(df):
    df.j_tot = df.j_CB + df.j_VB + np.nan_to_num(df.j_IB)

ITER = 0

#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=-0.2 m=mixed.out.d/po_thmeq.csv.0')
#df = pd.read_csv('/home/user/j/out.beforechange/diode c=1e18 V=0.4 m=mixed.out.d/final.csv.0')
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/progress.csv.{}'.format(ITER))
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/thmeq.csv.{}'.format(0))
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/final.csv.{}'.format(ITER))
#opt 8.000000.csv.0

df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/Vext V=-0.500000000.csv.{}'.format(ITER))
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/po_thmeq.csv.{}'.format(ITER))
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/opt inf.csv.{}'.format(ITER))
#df = pd.read_csv('/home/user/j/out/diode c=1e18 V=0.2 m=mixed.out.d/opt 8.000000.csv.{}'.format(ITER))

In [ ]:
# EXTRACT JV

from glob import glob
from parse import parse
import os

#WHERE = "save.out.light cv_only bias=0.5V X=1e-6 e60f4b6015c1c1a27673c"
#WHERE = "out.hack-ib_NI_1e14_X_1e-6"
#WHERE = "out"
WHERE = "out.dark JV 20180731"

WHERE = "/home/user/j/" + WHERE

JVlist = []
for fn in glob(WHERE+"/diode c=1e18 V=0.2 m=mixed.out.d/*.csv.0"):
    r = parse("Vext V={:f}.csv.0", os.path.basename(fn))
    if not r: continue
    V, = r
    df = pd.read_csv(fn)
    add_jtot(df)
    JVlist.append((V, np.median(df.j_tot.values)))
    
JV = pd.DataFrame(JVlist, columns='V J'.split())
JV.sort_values('V', inplace=True)

In [ ]:
#JV1 = JV
#JV2 = JV

In [ ]:
# PLOT JV

JV1 = JV1[(JV1.V <= 1.1) & (JV1.V > 0)]
JV2 = JV2[(JV2.V <= 1.1) & (JV2.V > 0)]
with subplots(closefig=0) as (fig, ax):
    ax.set_title(r"JV curve")
    ax.set_xlabel(r"Applied bias (V)")
    ax.set_ylabel(r"Current density ($\mathrm{\mu A/cm^2}$)")
    ax.plot(JV1.V, (abs(JV1.J)/1e3)*1e6, label=r"$N_I=2.5\times 10^{17}\;\mathrm{cm^{-3}}$")
    ax.plot(JV1.V, (abs(JV2.J)/1e3)*1e6, label=r"$N_I=10^{14}\;\mathrm{cm^{-3}}$")
    ax.set_yscale('log')
    ax.grid(True)
    ax.legend()

In [ ]:
df.IBmask = IBmask = np.where(df.actual_IB > 0.5, 1.0, np.nan)

df.mu_IB *= IBmask
df.j_IB *= IBmask
df.edgeE_IB *= IBmask

In [ ]:
#df.columns;0

df.u_VB.values[:8]

In [ ]:
D = df
#D = D[D.coord_x > 0.165]

#plt.plot(D['coord_x'], D.mu_CB)
with subplots(closefig=0) as (fig, ax):
    for t in 'ci iv cv'.split():
        k = 'alpha_'+t
        ax.plot(D['coord_x'], D[k], label=k)
    ax.set_title(r"extinction coeff, $\mathrm{1/cm}$")
    ax.grid(True)
    ax.legend()
    
with subplots(closefig=0) as (fig, ax):
    for t in 'ci iv cv'.split():
        k = 'I_'+t
        ax.plot(D['coord_x'], D[k], label=k)
    ax.set_title(r"I, $\mathrm{1/cm^2/s}$")
    ax.grid(True)
    ax.legend()

with subplots(closefig=0) as (fig, ax):
    for t in 'ci iv cv'.split():
        k = 'optg_'+t
        ax.plot(D['coord_x'], D[k], label=k)
    ax.set_title(r"optical generation, $\mathrm{1/cm^3}$")
    ax.grid(True)
    ax.legend()

In [ ]:
#w = df['w_base_CB']+df['w_delta_CB']
#w2 = df2['w_base_CB']+df2['w_delta_CB']

#plt.plot(df['coord_x'], w-w2)

D = df
#D = D[D.coord_x > 0.165]

#plt.plot(D['coord_x'], D.mu_CB)
with subplots(closefig=0) as (fig, ax):
    ax.plot(D['coord_x'], D.j_CB + D.j_VB + np.nan_to_num(D.j_IB))
    ax.set_title("j_total")
    ax.grid(True)

    #plt.plot(df['coord_x'], df2.w_base_CB)
    #plt.plot(df['coord_x'], w2)

with subplots(closefig=0) as (fig, ax):
    ax.plot(D['coord_x'], D.u_IB / 2.5e+17)
    ax.set_title("IB fill fraction")
    ax.set_xlim((1, 2))
    ax.grid(True)

In [ ]:
D=df
with subplots(closefig=0) as (fig, ax):
    for t in 'CB'.split():
        k = 'u_'+t
        ax.plot(D['coord_x'], D[k], label=k)
#    ax.plot(D['coord_x'], D['aeu_p'], label='aeu_p')
#    ax.plot(D['coord_x'], D['aeu_n'], label='aeu_n')
#    ax.plot(D['coord_x'], D['u_bc_CB_n_contact'], label='bc1')
    ax.plot(D['coord_x'], D['u2_bc_CB_n_contact'], label='bc2')
    #ax.plot(D['coord_x'], np.exp(D['V']*40), label='V')
    ax.set_title(r"u, $\mathrm{1/cm^3}$")
    ax.set_yscale('log')
    ax.grid(True)
    ax.legend()

In [ ]:
# checking whether u obeys its BC
D=df
with subplots(closefig=0) as (fig, ax):
    #ax.plot(D['coord_x'], D.u_bc_CB_p_contact)
    ax.plot(D['coord_x'], D.mu2_VB)
    ax.plot(D['coord_x'], D.mu2_CB)
    #ax.plot(D['coord_x'], D.mu2_VB)
    #ax.set_yscale('log')
    ax.set_title("u")
    ax.grid(True)

In [ ]:
# checking whether w obeys its BC
D=df
with subplots(closefig=0) as (fig, ax):
    ax.plot(D['coord_x'], D.w_bc_CB_p_contact, label="w_BC")
    ax.plot(D['coord_x'], D.w_base_CB + D.w_delta_CB, label="w")
    ax.set_title("w vs w_BC")
    ax.grid(True)
    ax.legend()

In [ ]:
add_jtot(df)

do_band_plot(None, df)
do_j_plot(None, df)